In [1]:
!pip install torch==2.5.1 torchvision==0.20.1

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com


In [2]:
import os

import torch
import torch.nn as nn
import torchvision

from glob import glob

from torch.utils.data.dataloader import default_collate
from torchvision import datasets, models, transforms

In [3]:
classes = [os.path.basename(folder) for folder in sorted(glob("../../tmp/test_data/*"))]

classes

['cloudy', 'others']

In [4]:
device = torch.device("cpu")

In [5]:
model = models.efficientnet_v2_m()

In [6]:
num_ftrs = model.classifier[1].in_features
model.classifier[1] = nn.Linear(num_ftrs, len(classes))

In [7]:
model.load_state_dict(torch.load("../../tmp/efficientnet_v2_m.pth", map_location=device, weights_only=False))

<All keys matched successfully>

In [8]:
data_transform = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor()
])

image_dataset = datasets.ImageFolder("../../tmp/test_data", data_transform)

dataloader = torch.utils.data.DataLoader(image_dataset, batch_size=4, shuffle=False, num_workers=2)

In [9]:
inputs, labels = next(iter(dataloader))

outputs = model(inputs)

outputs

tensor([[-2.4023e-01,  4.6065e-01],
        [-6.1739e-04, -7.0223e-02],
        [-1.6689e+00,  9.4089e-01],
        [ 1.7426e-01, -5.0595e-01]], grad_fn=<AddmmBackward0>)

In [10]:
_, preds = torch.max(outputs, 1)

preds

tensor([1, 0, 1, 0])

In [11]:
softmax = nn.Softmax(dim=1)

scores = softmax(outputs)

scores = torch.max(scores, 1).values

scores

tensor([0.6684, 0.5174, 0.9315, 0.6638], grad_fn=<MaxBackward0>)

In [12]:
predictions = []
for j in range(inputs.size()[0]):
    predictions.append({
        "correct_class": classes[labels[j]],
        "predicted_class": classes[preds[j]],
        "predicted_score": float(scores[j])
    })

predictions

[{'correct_class': 'cloudy',
  'predicted_class': 'others',
  'predicted_score': 0.6683817505836487},
 {'correct_class': 'cloudy',
  'predicted_class': 'cloudy',
  'predicted_score': 0.517394483089447},
 {'correct_class': 'cloudy',
  'predicted_class': 'others',
  'predicted_score': 0.9314864277839661},
 {'correct_class': 'cloudy',
  'predicted_class': 'cloudy',
  'predicted_score': 0.6637860536575317}]

In [13]:
%%time

predictions = []
for inputs, labels in dataloader:
    inputs = inputs.to(device)
    labels = labels.to(device)

    outputs = model(inputs)
    
    _, preds = torch.max(outputs, 1)
    scores = torch.max(softmax(outputs), 1).values

    for j in range(inputs.size()[0]):
        predictions.append({
            "correct_class": classes[labels[j]],
            "predicted_class": classes[preds[j]],
            "predicted_score": float(scores[j])
        })

predictions

CPU times: user 1min 7s, sys: 10.4 s, total: 1min 17s
Wall time: 31 s


[{'correct_class': 'cloudy',
  'predicted_class': 'others',
  'predicted_score': 0.5029999017715454},
 {'correct_class': 'cloudy',
  'predicted_class': 'others',
  'predicted_score': 0.5805233120918274},
 {'correct_class': 'cloudy',
  'predicted_class': 'others',
  'predicted_score': 0.9115483164787292},
 {'correct_class': 'cloudy',
  'predicted_class': 'cloudy',
  'predicted_score': 0.5687546133995056},
 {'correct_class': 'cloudy',
  'predicted_class': 'cloudy',
  'predicted_score': 0.7493675947189331},
 {'correct_class': 'cloudy',
  'predicted_class': 'cloudy',
  'predicted_score': 0.7625610828399658},
 {'correct_class': 'cloudy',
  'predicted_class': 'others',
  'predicted_score': 0.7457485198974609},
 {'correct_class': 'cloudy',
  'predicted_class': 'others',
  'predicted_score': 0.7063394784927368},
 {'correct_class': 'cloudy',
  'predicted_class': 'others',
  'predicted_score': 0.5802710056304932},
 {'correct_class': 'cloudy',
  'predicted_class': 'others',
  'predicted_score': 0